In [1]:
import os
from pathling import PathlingContext

MIMIC_FHIR_PATH = "/Users/szu004/datasets/work/mimic-iv/mimic4-ptl"

In [2]:
os.environ['SPARK_CONF_DIR'] = os.path.abspath('../env/spark-conf')
pc = PathlingContext.create()
spark = pc.spark
print('spark.sql.ansi.enabled', spark.conf.get("spark.sql.ansi.enabled"))
print('spark.sql.timestampType', spark.conf.get("spark.sql.timestampType"))
data = pc.read.parquet(MIMIC_FHIR_PATH)

:: loading settings :: url = jar:file:/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/szu004/.ivy2/cache
The jars for the packages stored in: /Users/szu004/.ivy2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cb4e1253-d72a-4c6d-b1a0-25ef409ba8cb;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT in local-m2-cache
	found io.delta#delta-spark_2.12;3.2.0 in local-m2-cache
	found io.delta#delta-storage;3.2.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
	found org.apache.hadoop#hadoop-aws;3.3.4 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in local-m2-cache
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
:: resolution report :: resolve 147ms :: artifacts dl 8ms
	:: modules in use:
	au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT from local-m2-cache in [default]
	co

2024-09-10 11:33:21 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-09-10 11:33:23 WARN  SimpleFunctionRegistry:72 - The function date_diff replaced a previously registered function.
spark.sql.ansi.enabled false
spark.sql.timestampType TIMESTAMP_LTZ


In [3]:
from sof import ViewCtx, SOFView, SQLView

In [4]:
import glob
VIEW_CTX = ViewCtx(spark, data)

for view_file in glob.glob('../src/sof/*.json'):
    sof_view = SOFView.from_file(view_file)
    sof_view.run(VIEW_CTX)

for view_file in sorted(glob.glob('../src/sql/*.sql')):
    sof_view = SQLView.from_file(view_file)
    sof_view.run(VIEW_CTX)

# SQLView.from_file('../src/sql/md_icustay_detail.sql').cache(VIEW_CTX)
# SQLView.from_file('../src/sql/md_oxygen_delivery.sql').cache(VIEW_CTX)

Creating SOF view rv_patient
2024-09-10 11:33:26 WARN  SparkStringUtils:72 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
Creating SOF view rv_o2_delivery_device
Creating SOF view rv_obs_o2_flow
Creating SOF view rv_icu_encounter
Creating SQL view dv_icu_encounter
Creating SQL view dv_o2_delivery_device
Creating SQL view dv_obs_o2_flow
Creating SQL view dv_patient
Creating SQL view md_icustay_detail
Creating SQL view md_oxygen_delivery
Creating SQL view st_subject
Creating SQL view stx_o2_flow
Creating SQL view coh_subject
Creating SQL view coh_reading_o2_flow


In [5]:
#spark.sql("CACHE TABLE md_icustay_detail")
#spark.sql("CACHE TABLE md_oxygen_delivery")

In [11]:
spark.sql("""DROP TABLE IF EXISTS md_icustay_detail_tbl;
CREATE TABLE md_icustay_detail_tbl AS SELECT * FROM md_icustay_detail""")

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'CREATE': extra input 'CREATE'.(line 2, pos 0)

== SQL ==
DROP TABLE IF EXISTS md_icustay_detail_tbl;
CREATE TABLE md_icustay_detail_tbl AS SELECT * FROM md_icustay_detail
^^^


In [7]:
spark.sql("SELECT * FROM md_icustay_detail_tbl").show()

+--------------------+--------------------+------+--------+-------------------+-------------------+--------------+---------------+
|          subject_id|             stay_id|gender|    race|          admittime|          dischtime|first_icu_stay|first_hosp_stay|
+--------------------+--------------------+------+--------+-------------------+-------------------+--------------+---------------+
|Patient/00110545-...|Encounter/bfd2cb4...|     M|   ASIAN|2184-05-12 13:19:46|2184-05-16 11:33:28|          true|           true|
|Patient/00194dab-...|Encounter/6f46559...|     F|   WHITE|2161-05-04 08:00:54|2161-05-08 04:02:24|          true|           true|
|Patient/0020080c-...|Encounter/a85f007...|     M|   WHITE|2163-02-01 14:56:00|2163-02-03 05:07:32|          true|           true|
|Patient/00250175-...|Encounter/4953a83...|     F|   WHITE|2170-01-05 02:21:53|2170-01-06 08:04:40|          true|           true|
|Patient/00296b15-...|Encounter/562620a...|     M|   WHITE|2117-10-03 13:08:36|2117